# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.8 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

1. Haszowanie zapewnia tylko *integrity*, ponieważ nie uwzględnia klucza prywatnego. Funkcja haszująca generuje unikalną wartość dla określonej wiadomości, a ta sama wiadomość zawsze daje ten sam wyniki.

2. Kody MAC nie zapewniają *non-repudiation*, ponieważ nie są w stanie określić, kto dokładnie utworzył i przesłał wiadomość. Odbiorca wiadomości oraz atakujący komunikację mogą dysponować kluczem prywatnym, który umożliwia tworzenie i weryfikowanie kodów MAC. W przypadku popularnego kodu HMAC, klucz prywatny jest znany tylko nadawcy i odbiorcy, ale nie jest w stanie określić, kto dokładnie użył klucza prywatnego w procesie generowania kodu MAC.

3. Podczas tworzenia podpisu cyfrowego, używany jest klucz prywatny, który tylko nadawca zna, co zapewnia *authentication*. Podczas procesu weryfikacji podpisu, odbiorca jest w stanie wykryć każdą zmianę w wiadomości, ponieważ zmiana ta zmieni wartość funkcji haszującej, co zapewnia *integrity*. Podpis cyfrowy zapewnia również *non-repudiation*, ponieważ proces weryfikacji podpisu wymaga klucza publicznego, który jest znany odbiorcy. To oznacza, że odbiorca ma dowód na to, że wiadomość została podpisana przez określonego nadawcę, a nadawca nie może zaprzeczyć, że podpisał tę wiadomość.

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [281]:
nlen = 2048
p, q, f, e, d, n = 0, 0, 0, 0, 0, 0

def find_primes():
  global p, q
  while(True):
    p = cu.getPrime(nlen//2)
    if (pow(2, (nlen-1)//2) < p) and (p < (1 << (nlen//2))):
      break
  while(True):
    q = cu.getPrime(nlen//2)
    if(q != p and (pow(2, (nlen-1)//2) < q) and (q < (1 << (nlen//2))) and (abs(p-q) > pow(2, (nlen/2)-100))):
      break

def public_key():
  global e
  while(True):
    e = cu.getRandomInteger(256)
    if(65537 <= e and e < pow(2, 256) and e < f and cu.GCD(f, e)==1 and (math.lcm(p-1, q-1) >= e * (1 << (nlen//2)))):
      break

def private_key():
  global d
  d = cu.inverse(e, f)

def key():
  global f, n
  find_primes()
  n = p * q
  f = (p-1)*(q-1)
  public_key()
  private_key()

In [282]:
key()
print(p)
print(q)
print(f)
print(e)
print(d)
print(n)

168874638401684103110524607753749399019212382618725261372821620942251364852465433329335549136494696571346626225817062685015033983003269740566600132720005749449480287037141444580493814634927542759623507894972431707880302529512184078262889672515763460272055046899496231463073644389463805419516000190863419115069
91323875402085178771550792636837811918863244315853186130341399239810956975087749848481487850199581117998005012104808795150895574434750836947411648434743514091091812890115715682618123346992393213236868083388328244303894777405919336781555806985632436761048510986178749760246203685973411270084026213434568759377
154222864359675879936748096769718205764548472114951694034852343209023738504723426933001372542363081796487520137766897454796981743809518953040089865788409644350003050863324334996507342014425949919508704580949173605770273467369886440943330696994084165352430136331701727491942928914110801490481168562239191905091532758864752721819814943894211424845227846054939334575609834226251949785

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [283]:
def string_to_int(msg):
    return int.from_bytes(msg.encode(), byteorder='big')

def int_to_string(num):
    length = math.ceil(num.bit_length() / 8)
    return num.to_bytes(length, byteorder='big').decode()

def encrypt(msg):
  M = string_to_int(msg)
  return pow(M, e, n)

def decrypt(C):
  M = pow(C, d, n)
  return int_to_string(M)

In [284]:
M = "Ala ma kota"

C = encrypt(M)
print(C)
print(decrypt(C))

13766290455959620611651480490456359447811946741766321612425972020234154970622274048780625570015369646062471696275619577658758594626728986451476369604903221375884531977486115323084529559857407544710912853406689571357699931054728858663019159739422950279369367632714862505244528078435062220961697148357112293801995443029666278141554000623908026623800625093281923263472371961453935981314451330793106769554083345284985481697073650476761727365518051309175320374070974666376604473394276988612828505437780458167405082061645910435597540780102373527140873866404384319337034276649720893395881572115603767769423551846665319838653
Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [287]:
from hashlib import sha3_256

def H(M):
  h = sha3_256(M)
  h = h.digest()
  return h

def digital_sign(M):
  m = bytearray()
  m.extend(map(ord, M))
  h = H(m)
  h = int.from_bytes(h, "big")
  SIG = pow(h, d, n)
  return SIG

def verify_signature(M, SIG):
  m = bytearray()
  m.extend(map(ord, M))
  h = H(m)
  h = int.from_bytes(h, "big")
  VER = pow(SIG, e, n)
  assert VER == h, "Wrong signature!"

In [288]:
M = 'Ala ma kota'

SIG = digital_sign(M)
print(SIG)
verify_signature(M, SIG)

9838944116951938196940776018703681982080700902520443155521101896803793513074246232232923096449844178753593373512616676782354890069741185244579200417967593561035256816266735561504919989768583232237927465828401257225013909793448234879114248345927203509929349664441190136327061272159295322977220695643370054902502906959608414525521189413404523675367085544872960644639179180199390590863709505384373004994392293829906191291065511768148900434167441350968087478027432420904604541127498827734937473238607350357010763023426552742618521258995370114296920592474681991621566505032098411561955865396809702423409719338318112030791


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [289]:
import string
def MGF1(M, len):
  hlen = 256//8
  output = b''
  if(type(M)==str):
    m = bytearray()
    m.extend(map(ord, M))
  else:
    m = M
  for i in range(int(math.ceil(len/hlen))):
    bytes_i = i.to_bytes(4, byteorder='big')
    tmp = H(b''.join([m, bytes_i]))
    output = output + tmp
  return output[:len]

In [290]:
M = 'Ala ma kota'
print(MGF1(M, 62))
print(len(MGF1(M, 62)))

b'\xf9c\xbdZ\x96\xed\x14\xb7\xf27\xd1\x15 Z\xc0\x08\x06)\x1b$\x13U;|$\x81\xa4\x1d\xad\x8e\xae\xd7#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4'
62


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

### Zadanie 6

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [291]:
import secrets

def encrypt_OAEP(M):
  hlen = 256//8
  k = nlen//8
  if(type(M)==str):
    m = bytearray()
    m.extend(map(ord, M))
  else:
    m = M
  mlen = len(M)
  assert mlen <= k - 2*hlen -2, 'Error'
  lHash = H(b'')
  PS = bytes(k - mlen - 2*hlen -2)
  DB = b''.join([lHash, PS, b'\x01', m])
  seed = secrets.token_bytes(hlen)
  dbMask = MGF1(seed, k-hlen-1)
  maskedDB = bytes([a ^ b for a, b in zip(DB, dbMask)])
  seedMask = MGF1(maskedDB, hlen)
  maskedSeed = bytes([a ^ b for a, b in zip(seed, seedMask)])
  EM = b''.join([b'\x00', maskedSeed, maskedDB])
  return EM

def decrypt_OAEP(EM):
  hlen = 256//8
  k = len(EM)
  Y = EM[:1]
  maskedSeed = EM[1:1+hlen]
  maskedDB = EM[1+hlen:]
  lHash = H(b'')
  seedMask = MGF1(maskedDB, hlen)
  seed = bytes([a ^ b for a, b in zip(maskedSeed, seedMask)])
  dbMask = MGF1(seed, k-hlen-1)
  DB = bytes([a ^ b for a, b in zip(maskedDB, dbMask)])
  lHash_ = DB[:hlen]
  
  x01 = DB.find(0x01, hlen)
  PS = DB[hlen:x01]
  assert DB[x01:x01+1] == b'\x01', 'Error 0x01'
  M = DB[x01+1:]

  # PS = DB[hlen:hlen+(k-mlen-2*hlen-2)]
  # assert DB[hlen+(k-mlen-2*hlen-2):hlen+(k-mlen-2*hlen-2)+1] == b'\x01', 'Error 0x01'
  # M = DB[hlen+(k-mlen-2*hlen-2)+1:]

  assert Y == b'\x00', 'Error Y'
  assert lHash == lHash_, 'Error lHash'
  return M

In [292]:
M = "Ala ma kota"
EM = encrypt_OAEP(M)
print(EM)
print(decrypt_OAEP(EM).decode("utf-8"))

b'\x00C\xb3NF-\x9dF\xb7z\x9b\x121\xb0N\xea\xbe\x1e\xee\xdd\x87.\x7fk\xafo\xc1\xec\x8c\xa4\xde\xd1\xf3\xfc\x11\xd1|\xa0[@\xea\x96\xe8\x84N\x80\xcd\xb9\xc4\xd1w/|\x92\xeb\xc3h\x99\xa8q;\x15\x8b\x98\x8e\xafn\x90\x91\x96\xa3\x916@\xa5G~\x14-(S\x0fj\xf0R\xbb^\x9e*>\x1c\x87\xf8\xec\xb5\xae|\x87}f\xd7b8S\xa863\xcd\xbaQ\x8e\x83hQ\xa2\xb0-\xa7m/\x91$\xa5\xa9_\xc4\xe0\x15*\x81b\xea\x95-b\x07\xe8#\x8a46\x9dY\xe1\xf5\xe0gq=]5\xcd\\\xf2\x91U\x98-\xdbE@f\x15\xc44}\xdeX\xf1\xf3X-5w\xd9\xa3O\xdb%m\x99O\x1bY\xd6\xb4I\x94W\xa6Q\xe2\x16\xe3\xc6\xf9K\xb5\x82\xcb3\x8e4\x11\x8em\xce\x8c\xe3Z\xd2p\xe6\xd5\x13%\xa4[~\xd2\xfd\xaa\xa8q~\xef\xcb\x9c\xf7"\xb3\xa4\x03\xd7X\x9f\xde\xc0%\x98\x88\xfcTVZ\x80\xa6W\xca\xe2\xcc\xd6\x11s\xab\x15'
Ala ma kota


In [293]:
def encrypt_RSA(M):
  if(type(M)==str):
    m = bytearray()
    m.extend(map(ord, M))
  else:
    m = M
  paddedM = encrypt_OAEP(m)
  paddedM = int.from_bytes(paddedM, "big")
  return pow(paddedM, e, n)

def decrypt_RSA(C):
  paddedM = pow(C, d, n)
  paddedM = paddedM.to_bytes(nlen//8, "big")
  M = decrypt_OAEP(paddedM)
  return M

In [294]:
M = "Ala ma kota"
C = encrypt_RSA(M)
print(C)
print(decrypt_RSA(C).decode('utf-8'))

933213298331913384375690079357416932397482414505509894931050907016813899048944910371664373851039350411504799840705394040808826398089631381561327309847284161790313902303587466487508759674008678458870974954613944054378616515849072615219431612676118926534367947560601876603080669969824196175508333801224021100492204817668984255370047554820697555129578753203036590204390579703413329585057576956236543448647484637701507545350179787716446405198510111809650091155457949688088458808130715958561392049128487780971165876591156966362218936786426167732884643940747482016695905180952525687433569940208527219000927859997508285309
Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


### EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [295]:
def encrypt_EMSA(M):
  hlen = 256//8
  slen = hlen
  emBits = nlen - 1
  emlen = nlen//8
  if(type(M)==str):
    m = bytearray()
    m.extend(map(ord, M))
  else:
    m = M
  mHash = H(m)
  assert emlen >= hlen + slen + 2, 'Error'
  salt = secrets.token_bytes(slen)
  M_ = b''.join([bytes(8), mHash, salt])
  mHash_ = H(M_)
  PS = bytes(emlen - slen - hlen - 2)
  DB = b''.join([PS, b'\x01', salt])
  dbMask = MGF1(mHash_, emlen-hlen-1)
  maskedDB = bytes([a ^ b for a, b in zip(DB, dbMask)])
  num_zero_bits = 8*emlen - emBits
  num_zero_bytes = (num_zero_bits + 7)//8
  mask = ((1 << num_zero_bits) - 1) << (8*num_zero_bytes - num_zero_bits)
  mask = mask.to_bytes(num_zero_bytes, byteorder='big')
  masked = maskedDB[:num_zero_bytes]
  masked = bytes([masked[i] & (~mask[i]) for i in range(num_zero_bytes)])
  maskedDB = masked + maskedDB[num_zero_bytes:]
  EM = b''.join([maskedDB, mHash_, b'\xbc'])
  return EM

In [296]:
def decrypt_EMSA(M, EM):
  hlen = 256//8
  slen = hlen
  emBits = nlen - 1
  emlen = nlen//8
  if(type(M)==str):
    m = bytearray()
    m.extend(map(ord, M))
  else:
    m = M
  mHash = H(m)
  assert emlen >= hlen + slen + 2, 'Error'
  assert EM[-1:] == b'\xbc', 'Error 0xbc'
  maskedDB = EM[:emlen-hlen-1]
  mHash_ = EM[emlen-hlen-1: emlen-1]

  num_zero_bits = 8*emlen - emBits
  num_zero_bytes = (num_zero_bits + 7)//8
  maskedDB_bits = ''.join([bin(byte)[2:].zfill(8) for byte in maskedDB[:num_zero_bytes]])
  for i in range(num_zero_bits):
    assert maskedDB_bits[i] == '0', 'Error O bits'
  dbMask=MGF1(mHash_,emlen-hlen-1)
  DB = bytes([a ^ b for a, b in zip(maskedDB, dbMask)])

  mask = ((1 << num_zero_bits) - 1) << (8*num_zero_bytes - num_zero_bits)
  mask = mask.to_bytes(num_zero_bytes, byteorder='big')
  masked = DB[:num_zero_bytes]
  masked = bytes([masked[i] & (~mask[i]) for i in range(num_zero_bytes)])
  DB = masked + DB[num_zero_bytes:]

  assert DB[:emlen-hlen-slen-2] == bytes(emlen-hlen-slen-2), 'Error DB'
  assert DB[emlen-hlen-slen-2:emlen-hlen-slen-1] == b'\x01', 'Error 0x01' 

  salt = DB[-slen:]
  M_ = b''.join([bytes(8), mHash, salt])
  mHash__ = H(M_)
  assert mHash_==mHash__, 'Wrong signature!'

In [297]:
def pbits(B):
  for byte in B:
    print(bin(byte)[2:].zfill(8), end=' ')
  print('')

In [298]:
M = 'Ala ma kota'

EM = encrypt_EMSA(M)
decrypt_EMSA(M, EM)

Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [299]:
def digital_sign_RSA(M):
  h = encrypt_EMSA(M)
  h = int.from_bytes(h, "big")
  SIG = pow(h, d, n)
  return SIG

def verify_signature_RSA(M, SIG):
  VER = pow(SIG, e, n)
  EM = VER.to_bytes(nlen//8, byteorder='big')
  decrypt_EMSA(M, EM)

In [300]:
M = 'Ala ma kota'

SIG = digital_sign_RSA(M)
verify_signature_RSA(M, SIG)

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$
u mnie encrypt_rsa

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [303]:
from Crypto.Random.random import getrandbits

def generate_sym_key():
  hlen = 256
  rand = getrandbits(nlen-2*hlen-2*8)
  rand = rand.to_bytes((rand.bit_length() + 7) // 8, byteorder='big')
  key = H(rand)
  # print(key)
  ciphered_key = encrypt_RSA(key)
  return ciphered_key

def decode_sym_key(ciphered_key):
  key = decrypt_RSA(ciphered_key)
  return key

In [304]:
ciphered_key=generate_sym_key()
print(ciphered_key)
print(decode_sym_key(ciphered_key))

4224006308891386855306576192226480846370615008031732277844190759222263553074923270361560563935988217043052369092122364217435327359346054741371044847113815074182174729449015486068467500802780666971132526035102815861989850057150913643853058388010437759212555678848754497527183405536471946117620937282406224377133807393241291902789448938757643003598983606679807906950924175482945898091876443707640559360139856047394170005590687022832467441186795651365524938925817718134166496128111636134022025827343593861647935399809874658348778659644657750322300023503771786225476796278802090206100196601924814794458489578354530409143
b'\x14\xa0\xcaR\x07\xdd\xe3z\xfd\xf0;/\x90\xd8\x8crb\x1c\x88v\x1fEK\x82p`\x87*\xef\xde\x85\x0b'
